# 00. Installation and configuration

In this example you will optionally create an Azure Machine Learning Workspace and initialize your notebook directory to easily use this workspace.  Typically you will only need to run this once per notebook directory, and all other notebooks in this directory or any sub-directories will automatically use the settings you indicate here.


## Prerequisites:

### 1. Install Azure ML SDK

**Before running this notebook, complete following steps. Then re-launch Jupyter notebooks.**

Install [miniconda](https://conda.io/miniconda.html) if you don't have conda Python distribution already.

Got to command line terminal window, and type

```shell
$ conda create -n myenv Python=3.6 cython numpy
```

On Windows, activate the environment:
```shell
$ conda activate myenv
```

If you are running Mac OS you should run
```shell
$ source activate myenv
```
Install azure-cli. This step can take several minutes.

```shell
(myenv) $ pip install azure-cli
```

Install or update azureml meta-package

```
(myenv) $ pip install --upgrade --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1 azureml-sdk[notebooks]
```

### 2. Install conda-aware Jupyter Notebook server, and run history widgets

```shell
# install Jupyter 
(myenv) $ conda install nb_conda

# install run history widget
(myenv) $ jupyter nbextension install --py --user azureml.train.widgets

# enable run history widget
(myenv) $ jupyter nbextension enable --py --user azureml.train.widgets
```

That's it! Now re-launch Jupyter notebooks from the new environment with Azure ML SDK.

```shell
(myenv) $ jupyter notebook
```

### Connect to your Azure Subscription

In order to use an AML Workspace, first you need access to an Azure Subscription.  You can [create your own](https://azure.microsoft.com/en-us/free/) or get your existing subscription information from the [Azure portal](https://portal.azure.com).

First login to azure and follow prompts to authenticate. Then check that your subscription is correct

In [1]:
!az login

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "id": "421bcd6b-87f3-4184-aa20-4eacf33f710c",
    "isDefault": false,
    "name": "Microsoft Azure Internal Consumption",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "jakazmie@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "5ffac217-ddd0-4388-8019-457477bd9d12",
    "isDefault": false,
    "name": "MTC SVC Team AIRS-Common",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "jakazmie@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "952a710c-8d9c-40c1-9fec-f752138cc0b3",
    "isDefault": true,
    "name": "MTC SVC Team AIRS-Cortana An

In [2]:
!az account show -o table

EnvironmentName    IsDefault    Name                                 State    TenantId
-----------------  -----------  -----------------------------------  -------  ------------------------------------
AzureCloud         True         MTC SVC Team AIRS-Cortana Analytics  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47


If you have multiple subscriptions and need to change the active one, you can use a command
```shell
az account set -s <subscription-id>
```

### Register Machine Learning Services Resource Provider

This step is required to use the Azure ML services backing the SDK.

In [ ]:
# register the new RP
!az provider register -n Microsoft.MachineLearningServices

# check the registration status
!az provider show -n Microsoft.MachineLearningServices

### Check core SDK version number for validate your installation and for debugging purposes

In [1]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

SDK Version: 0.1.4


## Initialize an Azure ML Workspace
### What is an Azure ML Workspace and why do I need one?

An AML Workspace is an Azure resource that organaizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an AML Workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.


### What do I need

To create or access an Azure ML Workspace, you will need to import the AML library and specify following information:
* A name for your workspace. You can choose one.
* Your subscription id. Use *id* value from *az account show* output above. 
* The resource group name. Resource group organizes Azure resources and provides default region for the resources in the group. You can either specify a new one, in which case it gets created for your Workspace, or use an existing one or create a new one from [Azure portal](https://portal.azure.com)

Please these values below. For workspace region, we prefer you use `eastus2` or `eastus2euap` (only if you have access to EUAP) for most scenarios. Other supported regions include `westcentralus`, `southeastasia`, `westeurope`, `australiaeast`, although their support might lag behind `eastus2` and `eastus2euap`.

In [3]:
subscription_id = "<my-subscription-id>"
resource_group = "myrg"
workspace_name = "myws"
workspace_region = "eastus2" # or eastus2euap

## Creating a workspace
If you already have access to an AML Workspace you want to use, you can skip this cell.  Otherwise, this cell will create an AML workspace for you in a subscription provided you have the correct permissions.

This will fail when:
1. The workspace already exists
2. You do not have permission to create a workspace in the resource group
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails for any reason other than already existing, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources.

**Note** The workspace creation can take several minutes.

In [4]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region)
ws.get_details()

{'id': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/rka0815/providers/Microsoft.MachineLearningServices/workspaces/myws',
 'name': 'myws',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'friendlyName': 'myws',
 'containerRegistry': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.containerregistry/registries/mywsacrfrlmcejk',
 'keyVault': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.keyvault/vaults/mywskeyvaulthyxpvigh',
 'applicationInsights': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.insights/components/mywsinsightskwibblnr',
 'batchaiWorkspace': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.batchai/workspaces/mywsbatchai_wsgntgrr',
 'identityPrincipalId': '18c78e1a-7efa-488f-9652-0ea39e5e8e09',
 'identityTen

## Configuring your local environment
You can validate that you have access to the specified workspace and write a configuration file to the default configuration location, `./aml_config/config.json`.

In [5]:
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Wrote the config file config.json to: C:\Users\roastala\notebooks\aml_config\config.json


You can then load the workspace from this config file from any notebook in the current directory.

In [6]:
# load workspace configuratio from ./aml_config/config.json file.ß
my_workspace = Workspace.from_config()
my_workspace.get_details()

Found the config file in: C:\Users\roastala\notebooks\aml_config\config.json


{'id': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/rka0815/providers/Microsoft.MachineLearningServices/workspaces/myws',
 'name': 'myws',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'friendlyName': 'myws',
 'containerRegistry': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.containerregistry/registries/mywsacrfrlmcejk',
 'keyVault': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.keyvault/vaults/mywskeyvaulthyxpvigh',
 'applicationInsights': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.insights/components/mywsinsightskwibblnr',
 'batchaiWorkspace': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/rka0815/providers/microsoft.batchai/workspaces/mywsbatchai_wsgntgrr',
 'identityPrincipalId': '18c78e1a-7efa-488f-9652-0ea39e5e8e09',
 'identityTen

## Create a folder to host all sample projects
Lastly, create a folder where all the sample projects will be hosted.

In [7]:
import os

sample_projects_folder = './sample_projects'

if not os.path.isdir(sample_projects_folder):
    os.mkdir(sample_projects_folder)
    
print('Sample projects will be created in {}.'.format(sample_projects_folder))

Sample projects will be created in ./sample_projects.


## Success!
Great, you are ready to move on to the rest of the sample notebooks.